In [ ]:
from pathlib import Path

import folium
import matplotlib.colors as mcolors
import numpy as np
from darts_acquisition import load_arcticdem, load_cdse_s2_sr_scene
from darts_preprocessing.engineering.arcticdem import calculate_aspect, calculate_hillshade

In [ ]:
s2id = "S2A_MSIL2A_20240823T060631_N0511_R134_T48XVJ_20240823T100146"
s2id = "S2C_MSIL2A_20250729T174931_N0511_R141_T17WMV_20250729T211514"
store = Path("./cdse-scenes")
scene = load_cdse_s2_sr_scene(s2id, store=store)
iroi = {"x": slice(5000, 6000), "y": slice(5000, 6000)}
scene = scene.isel(iroi)
scene

In [ ]:
adem_dir = Path("/isipd/projects/p_aicore_pf/darts-nextgen/aux/arcticdem_10m.icechunk")
adem = load_arcticdem(scene.odc.geobox, data_dir=adem_dir, resolution=10)
adem

In [ ]:
adem_nocoreg = adem.copy(deep=True)
adem_nocoreg = calculate_aspect(adem_nocoreg, debug=True)
adem_nocoreg = calculate_hillshade(
    adem_nocoreg, azimuth=scene.attrs["azimuth"], angle_altitude=scene.attrs["elevation"], debug=True
)
adem_nocoreg

In [ ]:
adem = calculate_aspect(adem)
adem = calculate_hillshade(adem, azimuth=scene.attrs["azimuth"], angle_altitude=scene.attrs["elevation"])
adem

In [ ]:
aspectcm = mcolors.ListedColormap(["red", "orange", "yellow", "green", "cyan", "blue", "purple", "magenta", "red"])
aspectcm

In [ ]:
hillshadecm = mcolors.ListedColormap(["black", "dimgray", "gray", "lightgray", "white"])
hillshadecm

In [ ]:
m = folium.Map(tiles="OpenStreetMap")
adem.dem.odc.explore(map=m, name="dem")
adem_nocoreg.aspect.odc.explore(map=m, vmin=0, vmax=360, cmap=aspectcm, name="aspect")
adem_nocoreg.hillshade.odc.explore(map=m, vmin=0, vmax=1, cmap=hillshadecm, name="hillshade")
adem.aspect.odc.explore(map=m, vmin=0, vmax=360, cmap=aspectcm, name="aspect (corrected)")
adem.hillshade.odc.explore(map=m, vmin=0, vmax=1, cmap=hillshadecm, name="hillshade (corrected)")
folium.LayerControl().add_to(m)
m.save("aspect_hillshade.html")
m

In [ ]:
x = adem.x.expand_dims({"y": adem.y})
y = adem.y.expand_dims({"x": adem.x})
correction_offset = np.arctan2(x, y) * (180 / np.pi) + 90
correction_offset[::20, ::20].plot()

In [ ]:
adem.dem.plot()

In [ ]:
adem.hillshade.plot()

In [ ]:
x = adem.x.max().item() - adem.x.min().item()
y = adem.y.max().item() - adem.y.min().item()
x = adem.x.mean()
y = adem.y.mean()
correction_offset = np.arctan2(x, y) * (180 / np.pi) + 90
print(correction_offset)
azimuth_corrected = (scene.attrs["azimuth"] - correction_offset + 360) % 360
azimuth_corrected

In [ ]:
scene.attrs["azimuth"]